In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import plot_model

In [2]:
data_folder = 'brain_tumor_final//'
train_dir = data_folder + "train//"
val_dir = data_folder + "val//"
test_dir = data_folder + "test//"

In [3]:
data_gen = ImageDataGenerator(rescale = 1./255,
                             shear_range = 0.2, 
                             zoom_range = 0.15,
                             rotation_range = 7,
                             width_shift_range=0.15,  
                             height_shift_range=0.1,  
                             horizontal_flip=True)

train_gen = data_gen.flow_from_directory(train_dir,
                                           target_size = (357, 328),
                                           batch_size = 32,
                                           shuffle = True,
                                           color_mode = 'grayscale',
                                            class_mode = 'binary')

val_test_gen = ImageDataGenerator(rescale = 1./255)

val_gen = val_test_gen.flow_from_directory(val_dir,
                                      target_size = (357, 328),
                                      batch_size = 32,
                                      shuffle = False,
                                      color_mode = 'grayscale',
                                      class_mode = 'binary')

test_gen = val_test_gen.flow_from_directory(test_dir,
                                      target_size = (357, 328),
                                      batch_size = 32,
                                      shuffle = False,
                                      color_mode = 'grayscale',
                                      class_mode = 'binary')

Found 2277 images belonging to 2 classes.
Found 488 images belonging to 2 classes.
Found 488 images belonging to 2 classes.


In [4]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3))
model.add(LeakyReLU())
model.add(MaxPool2D(pool_size=2, strides=2))

model.add(Conv2D(filters = 64, kernel_size=3))
model.add(LeakyReLU())
model.add(MaxPool2D(pool_size=2, strides=2))

model.add(Conv2D(filters = 128, kernel_size=3))
model.add(LeakyReLU())
model.add(MaxPool2D(pool_size=2, strides=2))

model.add(Conv2D(filters = 256, kernel_size=3))
model.add(LeakyReLU())
model.add(MaxPool2D(pool_size=2, strides=2))

model.add(Flatten())

model.add(Dense(units=256))
model.add(LeakyReLU())
model.add(Dropout(0.1))

model.add(Dense(units=1, activation='sigmoid'))

optimizer = Adam(lr=0.0001, decay=1e-5)

model.compile(optimizer = optimizer, 
            loss = 'binary_crossentropy', 
            metrics = ['accuracy'])

In [ ]:
stop = EarlyStopping(patience = 10, monitor = 'val_loss')
history = model.fit(x = train_gen, validation_data = val_gen, epochs = 100, shuffle = True, callbacks=[stop])

Epoch 1/100
72/72 [==============================] - 64s 834ms/step - loss: 0.6506 - accuracy: 0.5928 - val_loss: 0.4486 - val_accuracy: 0.7725
Epoch 2/100
72/72 [==============================] - 27s 372ms/step - loss: 0.5177 - accuracy: 0.7318 - val_loss: 0.4521 - val_accuracy: 0.7992
Epoch 3/100
72/72 [==============================] - 33s 459ms/step - loss: 0.4893 - accuracy: 0.7749 - val_loss: 0.4029 - val_accuracy: 0.8135
Epoch 4/100
72/72 [==============================] - 26s 360ms/step - loss: 0.4438 - accuracy: 0.7853 - val_loss: 0.4005 - val_accuracy: 0.8176
Epoch 5/100
72/72 [==============================] - 24s 336ms/step - loss: 0.4259 - accuracy: 0.8088 - val_loss: 0.3899 - val_accuracy: 0.8299
Epoch 6/100
72/72 [==============================] - 34s 466ms/step - loss: 0.4316 - accuracy: 0.8051 - val_loss: 0.4522 - val_accuracy: 0.7992
Epoch 7/100
72/72 [==============================] - 28s 392ms/step - loss: 0.4436 - accuracy: 0.7796 - val_loss: 0.3456 - val_accuracy:

In [ ]:
preds = model.predict(test_gen)

In [ ]:
preds = preds.round()

In [ ]:
preds

In [ ]:
labels = test_gen.labels

In [ ]:
accuracy_score(labels,preds)

In [ ]:
hist = pd.DataFrame(history.history)
plt.rcParams["figure.figsize"] = (20,10)
hist.plot()

In [ ]:
model.summary()

In [ ]:
# model.save('first_model.h5')